# Broadcasting

- The term broadcasting describes how NumPy treats arrays with different shapes during arithmetic operations.
- Subject to certain constraints, the smaller array is “broadcast” across the larger array so that they have compatible shapes. 
- Broadcasting provides a means of vectorizing array operations so that looping occurs in C instead of Python. 
- It does this without making needless copies of data and usually leads to efficient algorithm implementations. 
- There are, however, cases where broadcasting is a bad idea because it leads to inefficient use of memory that slows computation.


## Elementwise Operation

NumPy operations are usually done on pairs of arrays on an element-by-element basis. In the simplest case, the two arrays must have exactly the same shape, as in the following example:

In [1]:
import numpy as np
a = np.array([1.0, 2.0, 3.0])
b = np.array([2.0, 2.0, 2.0])
a * b

array([2., 4., 6.])

## Simplest Broadcasting Example

NumPy’s broadcasting rule relaxes this constraint when the arrays’ shapes meet certain constraints. The simplest broadcasting example occurs when an array and a scalar value are combined in an operation:

In [2]:
import numpy as np
a = np.array([1.0, 2.0, 3.0])
b = 2.0
a * b

array([2., 4., 6.])

## General Broadcasting Rules

### **Rule 1**: If the two arrays differ in their number of dimensions, the shape of the one with fewer dimensions is padded with ones on its leading (left) side.

```Python
Image  (3d array): 256 x 256 x 3
Scale  (1d array):             3
```
becomes
```Python
Image  (3d array):  256  x  256 x  3
Scale  (1d array): [256] x [256] x 3
Result (3d array):  256  x  256 x  3
```

Padding is indicated by []. **Note that this padding is effectively a view and does not require a memory copy.**

### **Rule 2**: If the shape of the two arrays does not match in any dimension, the array with shape equal to 1 in that dimension is stretched to match the other shape.

```Python
A      (4d array):  8 x 1 x 6 x 1
B      (3d array):  8 x 7 x 1 x 5
```
becomes
```Python
A      (4d array):   8  x [7] x  6  x 1
B      (3d array):   8 x  7 x [6] x 5
Result (4d array):   8  x  7 x  6  x 5
```

### **Rule 3**: If in any dimension the sizes disagree and neither is equal to 1, an error is raised.

```Python
A      (4d array):  8 x 1 x 6 x 1
B      (3d array):      7 x 2 x 5
```
gives an error.

## Example

In [3]:
import numpy as np

def generate_intersting_array(shape: tuple) -> np.ndarray:
    """
    Generate unique values based on index positions
    
    Parameters:
        shape (tuple): The shape of the desired N-dimensional array.
        
    Returns:
        numpy.ndarray: The generated array with unique hash values.
    """
    return np.fromfunction(
        lambda *indices: sum(shape[i + 1] * index for i, index in enumerate(indices[:-1])) + indices[-1],
        shape,
        dtype=int
    )


A = generate_intersting_array((10, 3, 8, 2, 5,  1))
B = generate_intersting_array((       8, 1, 5, 10))

print("Shape of product", (A*B).shape)
np.sum((B*A)-(A*B))

Shape of product (10, 3, 8, 2, 5, 10)


np.int64(0)

When the trailing dimensions of the arrays are unequal, broadcasting fails because it is impossible to align the values in the rows of the 1st array with the elements of the 2nd arrays for element-by-element addition.

In [4]:
A = generate_intersting_array((4, 3))
B = generate_intersting_array((   4))
try:
    A*B
except:
    pass

TypeError: 'int' object is not iterable

## Broadcasting and Matrix multiplication

Matrix multiplication is available using:
```Python
numpy.matmul
```
or
```Python
@
```

Numpy also provides:
```Python
np.dot
```

np.dot and np.matmul/@ have the same behavior when 2d arrays are used:

In [ ]:
A = generate_intersting_array((4, 3))
B = generate_intersting_array((3, 10))

m1 = A @ B
m2 = np.dot(A, B)
print("m1 shape:", m1.shape, "m2 shape:", m2.shape)
print("m1 == m2:", np.allclose(m1, m2,atol=0.0))

m1 shape: (4, 10) m2 shape: (4, 10)
m1 == m2: True


- For np.matmul:

If either argument is N-D, N > 2, it is treated as a stack of matrices residing in the last two indexes and broadcast accordingly.

- For np.dot:

For 2-D arrays it is equivalent to matrix multiplication, and for 1-D arrays to inner product of vectors (without complex conjugation). For N dimensions it is a sum product over the last axis of A and the second-to-last of B.

In [ ]:
A = generate_intersting_array((5, 3, 2, 4,  3))
B = generate_intersting_array((   3, 1, 3, 10))

m1 = A @ B
print("Shape of product", m1.shape)

m2 = np.dot(A, B)
print("Shape of product", m2.shape)

Shape of product (5, 3, 2, 4, 10)
Shape of product (5, 3, 2, 4, 3, 1, 10)


## The function to rule them all: np.einsum

Evaluates the Einstein summation convention on the operands.

### Einsum Subscripts Notation

- The subscripts string is a comma-separated list of subscript labels, where each label refers to a dimension of the corresponding operand (i, j, k, ...).
- Whenever a label is repeated it is summed.
- If a label appears only once, it is not summed.
- Repeated subscript labels in one operand take the diagonal.
- An implicit (classical Einstein summation) calculation is performed unless the explicit indicator ‘->’ is included as well as subscript labels of the precise output form.

### Transpose Example

$B_{ij} = A_{ji}$

In [ ]:
A = generate_intersting_array((4,3))
B = np.einsum('ij->ji', A)
print("Shape of A", A.shape, "Shape of B", B.shape)
print("A == B.T:", np.allclose(A, B.T, atol=0.0))

Shape of A (4, 3) Shape of B (3, 4)
A == B.T: True


### Matrix Multiplication Example

$M_{jk} = \sum_{k} A_{ik}B_{kj}$

In [ ]:
                            #  i,  k
A = generate_intersting_array((4,  3))
                            #  k,  l
B = generate_intersting_array((3, 10))

m1 = np.einsum('ik,kl->il', A, B)
m2 = A @ B
print("m1 shape", m1.shape, "m2 shape", m2.shape)
print("m1 == m2:", np.allclose(m1, m2,atol=0.0))

m1 shape (4, 10) m2 shape (4, 10)
m1 == m2: True


array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

### Broadcasting Example + Matrix Multiplication

In [ ]:

A = generate_intersting_array((5, 8, 3, 4, 3))
B = generate_intersting_array((   8, 1, 3, 4))

m1 = np.einsum('ijklm,jkmn->ijkln', A, B)
m2 = np.einsum('...lm,...mn->...ln', A, B)
m3 = A @ B
print("m1 shape", m1.shape, "m2 shape", m2.shape, "m3 shape", m3.shape)
print("m1 == m2:", np.allclose(m1, m2,atol=0.0), ", m1 == m3:", np.allclose(m1, m3,atol=0.0))


m1 shape (5, 8, 3, 4, 4) m2 shape (5, 8, 3, 4, 4) m3 shape (5, 8, 3, 4, 4)
m1 == m2: True , m1 == m3: True


### Trace Example

In [ ]:
A = generate_intersting_array((4, 4))
A_trace = np.einsum('ii->', A)
print("A_trace.shape", A_trace.shape)
print("A_trace == np.trace(A):", np.allclose(A_trace, np.trace(A), atol=0.0))

A_trace.shape ()
A_trace == np.trace(A): True
[ 0  5 10 15] 30


### Transpose Example

In [ ]:
A = generate_intersting_array((4, 10))
A_transpose = np.einsum('ij->ji', A)
print("A.shape", A.shape, "A_transpose.shape", A_transpose.shape)
print("A == A_transpose.T:", np.allclose(A, A_transpose.T, atol=0.0))

A.shape (4, 10) A_transpose.shape (10, 4)
A == A_transpose.T: True


## Phase shift Math Example: np.einsum

In [ ]:
time = 100000
rotmat_new_phase_direction = generate_intersting_array((3,3)) #[3 x 3]
rotmat_field_phase_direction = generate_intersting_array((time, 3, 3))  #[3]

# (3,3)@(time, 3, 3) -> (time, 3, 3) and transpose per time step.

uvw_rotmat_1 = np.matmul(rotmat_new_phase_direction, rotmat_field_phase_direction).transpose([0,2,1]) 
 
uvw_rotmat_2 = np.einsum('ij,tjk->tik', rotmat_new_phase_direction, rotmat_field_phase_direction).transpose([0,2,1]) 
 
uvw_rotmat_3 = np.einsum("ij,tjk->tki", rotmat_new_phase_direction, rotmat_field_phase_direction) 

print("uvw_rotmat_1.shape", uvw_rotmat_1.shape, "uvw_rotmat_2.shape", uvw_rotmat_2.shape, "uvw_rotmat_3.shape", uvw_rotmat_3.shape)
print("uvw_rotmat_1 == uvw_rotmat_2:", np.allclose(uvw_rotmat_1, uvw_rotmat_2, atol=0.0), ", uvw_rotmat_1 == uvw_rotmat_3:", np.allclose(uvw_rotmat_1, uvw_rotmat_3, atol=0.0))


uvw_rotmat_1.shape (100000, 3, 3) uvw_rotmat_2.shape (100000, 3, 3) uvw_rotmat_3.shape (100000, 3, 3)
uvw_rotmat_1 == uvw_rotmat_2: True , uvw_rotmat_1 == uvw_rotmat_3: True


In [ ]:
time = 100
baseline = 1000
uvw = generate_intersting_array((time, baseline, 3))
phase_rotation = generate_intersting_array((time, 3))

# Use None to add singleton dimension to phase_rotation: (time, 3) -> (time, 1, 3, 1) 
phase_direction_1 = np.matmul(uvw, phase_rotation[..., None]).squeeze(-1)
print(np.matmul(uvw, phase_rotation[..., None]).shape, phase_rotation[:, None, :, None].shape)

phase_direction_2 = np.einsum(
    " ijk, ik -> ij",
    uvw, phase_rotation
)# (time, baseline, 3), (time, 3) -> (time, baseline)

print("phase_direction_1.shape", phase_direction_1.shape, "phase_direction_2.shape", phase_direction_2.shape)
print("phase_direction_1 == phase_direction_2:", np.allclose(phase_direction_1, phase_direction_2, atol=0.0))

(100, 1000, 1) (100, 1, 3, 1)
phase_direction_1.shape (100, 1000) phase_direction_2.shape (100, 1000)
phase_direction_1 == phase_direction_2: True
